<a href="https://colab.research.google.com/github/jorgeturriate/VideoSegmentationWKallman/blob/main/VideoSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Segmentation

Firt, we need to perform object detection of the Tennis ball to use it as an input for the Kalman filter which will use these detections to predict the trayectory of the Tennis ball.
I tried first with Yolo v8, but the results using YoloV5 were better, so I changed to YoloV5.

In [ ]:
!pip install torch torchvision
!pip install git+https://github.com/ultralytics/yolov5.git

## Importing library

In [ ]:
#from ultralytics import YOLO
import cv2
from google.colab import drive
from google.colab.patches import cv2_imshow
import torch

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')
video_path = "/content/drive/My Drive/VideoSegmentation/RogerVideo.mp4"  # Update this with your actual video path

In [ ]:
output_video_path = "/content/drive/My Drive/VideoSegmentation/segmented_video_output.mp4"

# Load YOLOv5 model (you can use 'yolov5s' or a different model depending on your needs)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Pre-trained YOLOv8 Nano model

I got the HSV values for the Tennis ball and I'm using these values to set the range of the HSV values to improve the detection of the YOLOv5.

In [ ]:
import numpy as np

# Define the HSV range for the tennis ball (you can fine-tune these values)
lower_hsv = np.array([80, 50, 150])  # Lower bound for greenish tennis balls
upper_hsv = np.array([100, 255, 255])  # Upper bound for greenish tennis balls

Checking the initial features of the video.

In [ ]:
# Open video
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(f"Original Video Properties -> Width: {frame_width}, Height: {frame_height}, FPS: {fps}")

Defining the output video settings

In [ ]:
# Output video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

Initializing Kalman filter to follow the trayectory of the ball.

In [ ]:
# Initialize Kalman Filter
kalman = cv2.KalmanFilter(4, 2)  # 4 states (x, y, vx, vy), 2 measurements (x, y)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 0.03


Segmenting the video, we can see also the velocity of the ball inside the frames we are creating and the trayectory the ball was following by using the predictions of the Kalman filter.

In [ ]:
frame_count = 0
trajectory = []  # To store the trajectory of the ball
initial_kalman_initialized = False
last_position = None
last_time = 0
speed = 0  # Initial speed

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Convert frame to HSV for color-based segmentation
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create mask for tennis ball in HSV range
    mask = cv2.inRange(hsv_frame, lower_hsv, upper_hsv)

    # Perform YOLOv5 detection on the frame
    results = model(frame)

    # Process YOLOv5 detections
    detected_ball = None
    for result in results.xywh[0]:  # Format: [x_center, y_center, width, height, confidence, class_id]
        x_center, y_center, w, h, conf, cls = result.tolist()
        if int(cls) == 32:  # Replace with the class ID for 'tennis ball' (check with your model)
            detected_ball = (int(x_center), int(y_center))

    # Initialize Kalman Filter with the first detected ball position
    if detected_ball and not initial_kalman_initialized:
        kalman.statePost = np.array([detected_ball[0], detected_ball[1], 0, 0], np.float32)  # Initialize at first detected position
        initial_kalman_initialized = True

    # If the ball is detected, update Kalman Filter
    if detected_ball:
        kalman.correct(np.array([detected_ball[0], detected_ball[1]], np.float32))
        predicted = kalman.predict()
        predicted_x, predicted_y = int(predicted[0]), int(predicted[1])

        # Calculate speed (in pixels per frame)
        if last_position:
            dt = 1 / fps  # Time difference between frames (in seconds)
            dx = predicted_x - last_position[0]
            dy = predicted_y - last_position[1]
            speed = np.sqrt(dx**2 + dy**2) / dt  # Speed in pixels per second

        # Draw the predicted position
        cv2.circle(frame, (predicted_x, predicted_y), 10, (0, 0, 255), -1)
        cv2.putText(frame, f"Predicted Position", (predicted_x - 20, predicted_y - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.putText(frame, f"Speed: {speed:.2f} px/s", (predicted_x - 20, predicted_y + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    else:
        predicted = kalman.predict()
        predicted_x, predicted_y = int(predicted[0]), int(predicted[1])

    # Add the predicted position to the trajectory list
    trajectory.append((predicted_x, predicted_y))

    # Draw the trajectory (line through all previous positions)
    if len(trajectory) > 1:
        for i in range(1, len(trajectory)):
            cv2.line(frame, trajectory[i-1], trajectory[i], (255, 0, 0), 2)  # Blue line for trajectory

    # Write the processed frame (with both YOLO and Kalman Filter tracking)
    out.write(frame)

    # Optionally show the frame for debugging
    cv2_imshow(frame)  # Use cv2_imshow in Colab instead of cv2.imshow()

    last_position = (predicted_x, predicted_y)
    frame_count += 1

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# If you want to save the final image showing the trajectory, you can use the last frame or any frame you like
if frame is not None:
    final_frame = frame.copy()
    # Optionally, save a single frame showing the trajectory
    cv2.imwrite('/content/drive/MyDrive/tennis_ball_trajectory_final_v5.jpg', final_frame)

print(f"Processed {frame_count} frames. Output video saved to: {output_video_path}")